In [1]:
import pandas as pd

## Penjelasan Dataset, EDA dan Proses Features Dataset

Pada eksperiment kali ini saya menggunakan dua dataset yang berbeda, yang memiliki perbedaan jumlah record/data yang ada.
Dataset yang terlampir memiliki beberapa atribute seperti Nama Warna, Hex Color Code, RGB, Hue, Saturation(HSL.S), dan juga Lightness(HSL.L).
Atribute yang ada akan digunakan untuk mengklasifikasikan warna berdasarkan musim 'Seasonal Color' => Spring, Summer, Winter, Autumn.

In [2]:
#muat dataset colors
colors_df1 = pd.read_csv('colorhexa_com.csv')
colors_df1.info()
colors_df1.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 746 entries, 0 to 745
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           746 non-null    object 
 1   Hex (24 bit)   746 non-null    object 
 2   Red (8 bit)    746 non-null    int64  
 3   Green (8 bit)  746 non-null    int64  
 4   Blue (8 bit)   746 non-null    int64  
 5   Hue (degrees)  746 non-null    float64
 6   HSL.S (%)      746 non-null    float64
 7   HSL.L (%)      746 non-null    float64
dtypes: float64(3), int64(3), object(2)
memory usage: 46.8+ KB


,Name,Hex (24 bit),Red (8 bit),Green (8 bit),Blue (8 bit),Hue (degrees),HSL.S (%),HSL.L (%)
0,Air Force blue,#5d8aa8,93,138,168,204.0,30.1,51.2
1,Alice blue,#f0f8ff,240,248,255,208.0,100.0,97.1
2,Alizarin crimson,#e32636,227,38,54,354.9,77.1,52.0
3,Almond,#efdecd,239,222,205,30.0,51.5,87.1
4,Amaranth,#e52b50,229,43,80,348.1,78.2,53.3
5,Amber,#ffbf00,255,191,0,44.9,100.0,50.0
6,American rose,#ff033e,255,3,62,346.0,100.0,50.6
7,Amethyst,#9966cc,153,102,204,270.0,50.0,60.0
8,Android Green,#a4c639,164,198,57,74.5,55.3,50.0
9,Anti-flash white,#f2f3f4,242,243,244,210.0,8.3,95.3


In [3]:
#muat dataset colors
colors_df2 = pd.read_csv('wikipedia_color_names.csv')
colors_df2.info()
colors_df2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1298 entries, 0 to 1297
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Name                             1298 non-null   object 
 1   Hex (24 bit)                     1298 non-null   object 
 2   Red (8 bit)                      1298 non-null   int64  
 3   Green (8 bit)                    1298 non-null   int64  
 4   Blue (8 bit)                     1298 non-null   int64  
 5   Hue (degrees)                    1298 non-null   float64
 6   HSL.S (%)                        1298 non-null   float64
 7   HSL.L (%), HSV.S (%), HSV.V (%)  1298 non-null   float64
dtypes: float64(3), int64(3), object(2)
memory usage: 81.3+ KB


,Name,Hex (24 bit),Red (8 bit),Green (8 bit),Blue (8 bit),Hue (degrees),HSL.S (%),"HSL.L (%), HSV.S (%), HSV.V (%)"
0,Absolute zero,#0048BA,0,72,186,217.0,100.0,37.0
1,Acid green,#B0BF1A,176,191,26,65.0,76.0,43.0
2,Aero,#7CB9E8,124,185,232,206.0,70.0,70.0
3,Aero blue,#C9FFE5,201,255,229,151.0,100.0,89.0
4,African violet,#B284BE,178,132,190,288.0,31.0,63.0
5,Air Force blue (RAF),#5D8AA8,93,138,168,204.0,30.0,51.0
6,Air Force blue (USAF),#00308F,0,48,143,220.0,100.0,28.0
7,Air superiority blue,#72A0C1,114,160,193,205.0,39.0,60.0
8,Alabama crimson,#AF002A,175,0,42,346.0,100.0,34.0
9,Alabaster,#F2F0E6,242,240,230,346.0,32.0,93.0


### Proses Features Dataset (Features Engineering)
Mengkategorikan warna berdasarkan nilai RGB, Hue, HSL.S, dan HSL.L ke dalam 'Season'

In [4]:
def categorize_color_by_season(row):
    """
    Mengkategorikan warna berdasarkan nilai RGB, Hue, HSL.S, dan HSL.L ke dalam musim.

    Args:
    row (pd.Series): Baris dari DataFrame yang berisi nilai RGB, Hue, HSL.S, dan HSL.L.

    Returns:
    str: Kategori musim dari warna (Spring, Summer, Autumn, Winter).
    """
    r, g, b = row['Red (8 bit)'], row['Green (8 bit)'], row['Blue (8 bit)']
    hue = row['Hue (degrees)']
    saturation = row['HSL.S (%)']
    lightness = row['HSL.L (%), HSV.S (%), HSV.V (%)']
    
    # Menghitung kecerahan (brightness)
    brightness = (r + g + b) / 3
    
    # Menghitung kehangatan (warmth)
    warmth = r - b
    
    if lightness > 70:  # Warna terang
        if warmth > 20 and saturation > 50:
            return 'Spring'
        elif saturation < 50:
            return 'Summer'
    else:  # Warna gelap
        if warmth < -20 and saturation > 50:
            return 'Winter'
        elif warmth > 20:
            return 'Autumn'
    
    return 'Uncategorized'

## Proses Learning / Modeling + Performa Model

Pada proses ini dapat dilihat pada masing-masing file:
* [File dengan dataset colorhexa_com.csv](new_formulas_colors.ipynb)
* [File dengan dataset wikipedia_color_names.csv](colors_another_data.ipynb)

## Kemungkinan Overfitting
Model Decision Tree dengan skor hampir sempurna di semua fold menunjukkan potensi overfitting, terutama jika data validasi atau uji lainnya tidak tersedia untuk menguji generalisasi model. Jika model terlalu cocok dengan data pelatihan, itu mungkin tidak berkinerja baik pada data baru yang tidak terlihat.
Model Pertama dengan skor yang lebih rendah dan variasi di antara fold mungkin lebih generalizable, tetapi masih perlu diuji lebih lanjut.

## Solusi untuk Overfitting:
1. Regularisasi: Terapkan teknik regularisasi untuk mengurangi kompleksitas model.
2. Pruning (untuk Decision Tree): Gunakan pruning untuk mengurangi ukuran pohon keputusan dan menghindari overfitting.
3. Tambah Data Pelatihan: Gunakan lebih banyak data untuk melatih model, jika memungkinkan.
4. Validasi dengan Data Baru: Selalu validasi model dengan dataset yang benar-benar baru yang tidak pernah dilihat oleh model.

Experiment ini kemungkinan masih akan terus dikembangkan untuk melakukan eksperimen berikutnya, yakni merekomendasikan notes-parfume yang kemungkinan akan cocok dipadukan dengan 'Seasonal color' tertentu. Latihan experiment ini terdapat pada file: [new_formulas_colors.ipynb](new_formulas_colors.ipynb)